## Setup Docker container for DB and install lib

In [ ]:
!pip install -qU langchain_postgres
# run postgres pg_vector docker with command: docker compose up

## Setup Postgres pg_vector Vector Store and function for importing documents

In [6]:
import os
from app.db.VectorStore import VectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from tqdm import tqdm
from app.db.parser.FileContentProvider import FileContentProvider
from app.db.DocumentMetadataCreator import DocumentMetadataCreator

def import_source_documents(folder:str, verbose:bool=True, only_php:bool = False):
    # Split documents and store in vector db
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=10000,
        chunk_overlap=200
    )

    vectordb = VectorStore.get_vector_store(folder)
    dir = os.path.join("../data/", folder)
    document_metadata_creator = DocumentMetadataCreator()
    file_content_provider = FileContentProvider()
    for root, d_names, f_names in os.walk(dir):
        if verbose:
            progress_bar = tqdm(f_names)
        else:
            progress_bar = f_names
        for file in progress_bar:
            if verbose:
                progress_bar.set_description(file)
            if not os.path.isdir(file):
                text = file_content_provider.get_content(root, file, only_php=only_php)

                splits = []
                metadata = document_metadata_creator.create_metadata(root, file, text)
                for idx, chunk in enumerate(text_splitter.split_text(str(text))):
                    metadata['chunk_idx'] = idx
                    splits.append(Document(
                        page_content = chunk,
                        metadata = metadata
                    ))
                try:
                    vectordb.add_documents(splits)
                except Exception as e:
                    print(e)

                # TODO create code graph afterwards

## Load Documentation

In [2]:
vectordb = VectorStore.get_vector_store('documentation')

2025-02-02 07:00:18.237 
  command:

    streamlit run /home/michal/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
# drop tables
vectordb.drop_tables()

In [ ]:
import_source_documents('documentation')

## Load Codebase

In [ ]:
import_source_documents('code', verbose=False)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

'None'